In [4]:
import numpy as np
import cvxpy as cp

def kernel(x, y, sigma=1.0):
    # Example: RBF kernel
    return np.exp(-np.sum((x - y)**2) / (2.0 * sigma**2))

def build_gram_matrix(X, sigma=1.0):
    n = len(X)
    K = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            K[i, j] = kernel(X[i], X[j], sigma=sigma)
    return K

def kernel_quantile_regression(X, y, tau=0.5, lamb=1.0, sigma=1.0):
    """
    Solve:
      min_alpha  sum_i pinball(y_i - K[i, :] @ alpha) + lamb * alpha.T K alpha
    """
    K = build_gram_matrix(X, sigma)
    alpha = cp.Variable(len(X))

    # Pinball loss
    residuals = y - cp.psd_wrap(K) @ alpha
    # piecewise linear: rho_tau(u) = tau * u+ + (tau-1) * u-
    # we can implement with maximum of two linear expressions:
    #    rho_tau(u) = max( tau*u, (tau-1)*u )
    # But more explicitly, we typically do:
    #    rho_tau(u) = tau * cp.pos(u) + (tau - 1) * cp.pos(-u)

    loss = tau * cp.sum(cp.pos(residuals)) + (tau - 1) * cp.sum(cp.pos(-residuals))
    reg_term = lamb * cp.quad_form(alpha, K)
    objective = cp.Minimize(loss + reg_term)

    prob = cp.Problem(objective)
    prob.solve()

    return alpha.value

# Example usage on a small synthetic dataset
if __name__ == "__main__":
    np.random.seed(0)
    X = np.random.randn(50, 1)
    y = X.squeeze()**2 + 0.1*np.random.randn(50)  # some non-linear function

    alpha_hat = kernel_quantile_regression(X, y, tau=0.5, lamb=1.0, sigma=0.5)

    # Once alpha_hat is found, you can predict on new points x_new by:
    #   f(x_new) = sum_j alpha_hat[j] * kernel(X[j], x_new)


ArpackNoConvergence: ARPACK error -1: ARPACK error -1: No convergence (501 iterations, 0/1 eigenvectors converged)


        CVXPY note: This failure was encountered while trying to certify
        that a matrix is positive semi-definite (see [1] for a definition).
        In rare cases, this method fails for numerical reasons even when the matrix is
        positive semi-definite. If you know that you're in that situation, you can
        replace the matrix A by cvxpy.psd_wrap(A).

        [1] https://en.wikipedia.org/wiki/Definite_matrix
        

!pip install cvxpy

!pip install cvxpy

In [ ]:
!pip install cvxpy